# Define data/result directory

In [1]:
import os

data_folder = "../data"
result_folder = "../results"
manga_list = "vi/Ruri Dragon (Oneshot)/Ch. None"  # Will be changed to list later

# Ruri Dragon
manga_folder = os.path.join(data_folder, manga_list)

individual_result_folder = os.path.join(result_folder, manga_list)
json_output_dir = os.path.join(individual_result_folder, "json_results")
result_image_output_dir = os.path.join(individual_result_folder, "image_results")

cut_texts_dir = os.path.join(individual_result_folder, "cut_texts")

raw_images = os.listdir(manga_folder)
json_files = os.listdir(json_output_dir)
easy_ocr_viet_ocr_result_dir = os.path.join(
    individual_result_folder, "easy_ocr_viet_ocr_result"
)
os.makedirs(easy_ocr_viet_ocr_result_dir, exist_ok=True)

# Initialize models

In [2]:
import easyocr
from vietocr.tool.config import Cfg

# Initialize EasyOCR (Vietnamese)
reader = easyocr.Reader(["vi"])

config = Cfg.load_config_from_name("vgg_transformer")  # Load YAML config file
from vietocr.tool.predictor import Predictor

config["weights"] = "../models/viet_ocr/pretrained_weight/vgg_transformer.pth"
# config['weights'] = "../models/viet_ocr/custom_weight/transformerocr-1.pth"
config["cnn"]["pretrained"] = False
detector = Predictor(config)

d:\miniconda3\envs\easyocr-vietocr\lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Experimental function code, havent test yet

import os
import json
from PIL import Image


def get_transcript_from_image(img_path, reader, detector):
    """
    Run OCR pipeline on a single cropped image:
    1. Use EasyOCR to detect line regions
    2. Use VietOCR to recognize each cropped line
    3. Return merged transcript (string)
    """
    results = reader.readtext(img_path, detail=1, paragraph=False)

    if not results:
        return ""  # no text detected

    line_texts = []
    for coords, _, _ in results:  # coords = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
        x_min = min([pt[0] for pt in coords])
        y_min = min([pt[1] for pt in coords])
        x_max = max([pt[0] for pt in coords])
        y_max = max([pt[1] for pt in coords])

        img = Image.open(img_path).convert("RGB")
        cropped = img.crop((x_min, y_min, x_max, y_max))

        try:
            text_viet = detector.predict(cropped)
        except Exception as e:
            print(f"⚠️ VietOCR failed on {img_path}: {e}")
            text_viet = ""

        if text_viet.strip():
            line_texts.append(text_viet)

    return " ".join(line_texts)


def process_json_file(
    json_file,
    json_output_dir,
    cut_texts_dir,
    easy_ocr_viet_ocr_result_dir,
    reader,
    detector,
):
    """
    Process one JSON file:
    - Load original annotations
    - Replace essential text OCR results with EasyOCR+VietOCR pipeline output
    - Save updated JSON to result folder
    """
    base_name = os.path.splitext(json_file)[0]
    json_path = os.path.join(json_output_dir, json_file)
    cut_page_dir = os.path.join(cut_texts_dir, base_name)

    if not os.path.exists(cut_page_dir):
        print(f"⚠️ No cut_texts found for {json_file}, skipping...")
        return

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    new_ocr = []
    for idx, (bbox, is_essential) in enumerate(
        zip(data["texts"], data["is_essential_text"])
    ):
        if not is_essential:
            new_ocr.append(data["ocr"][idx])
            continue

        cut_img_path = os.path.join(cut_page_dir, f"{base_name}_{idx:03}.png")
        if not os.path.exists(cut_img_path):
            print(f"⚠️ Missing cut image {cut_img_path}, keeping original OCR.")
            new_ocr.append(data["ocr"][idx])
            continue

        try:
            vi_text = get_transcript_from_image(cut_img_path, reader, detector)
            if not vi_text:
                vi_text = data["ocr"][idx]
        except Exception as e:
            print(f"❌ OCR pipeline failed for {cut_img_path}: {e}")
            vi_text = data["ocr"][idx]

        new_ocr.append(vi_text)

    data["ocr"] = new_ocr

    out_path = os.path.join(easy_ocr_viet_ocr_result_dir, json_file)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"✅ Saved result for {json_file} → {out_path}")


for json_file in json_files:
    if json_file.endswith(".json"):
        process_json_file(
            json_file,
            json_output_dir,
            cut_texts_dir,
            easy_ocr_viet_ocr_result_dir,
            reader,
            detector,
        )

✅ Saved result for 04.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\04.json


Individual test

In [9]:
import os

test_image_dir = "../data/vi/test_data/multiple_line"
test_images = os.listdir(test_image_dir)

for test_image in test_images:
    img_path = os.path.join(test_image_dir, test_image)

    # 🔄 Reuse the function
    final_text = get_transcript_from_image(img_path, reader, detector)

    print(f"{test_image}: {final_text if final_text else '(empty)'}")

03_006.png: Ừ, TRÔNG CON GIỐNG HỆT BỐ CON ĐẤY
04_001.png: VÌ CON LÀ CON LAI GIỮA NGƯỜI VÀ RỒNG.
05_004.png: BÌNH TĨNH NÃO, ĐẦU PHẢI NÓ SẼ GIẾT CON HAY GÌ ĐÂU
vnexpress.png: Một phần nguyên nhân khiến Mỹ không năm được vụ Israel tập kích lãnh thổ Qatar là do tập trung giám sát nơi khác, theo quan chức CENTCOM. "Đòn tấn công của Israel nhằm vào mục tiêu Hamas ở Qatar hoàn toàn không có dấu hiệu hay cảnh báo trước, vì toàn bộ phương tiện giám sát và sự chú ý của chúng tôi đều không nằm ở đó. Không ai nghĩ điều này sẽ diễn ra", tướng Derek France, chỉ huy lực lượng không quân thuộc Bộ tư lệnh Trung tâm Mỹ (CENTCOM), cho biết hôm 24/9. CENTCOM là đơn vị đặc trách toàn bộ hoạt động của quân đội Mỹ tại Trung Đông. Phát biểu được tướng France đưa ra trong buổi thảo luận về hệ quả của cuộc tập kích của Israel nhảm vào thủ đô Doha của Qatar hồi đầu tháng, Sự việc khiến nhiều người đặt câu hỏi tại sao lưới phòng không, cảm biến tiên tiến của Mỹ và Qatar không phát hiện và đánh chặn mối đe dọa. Tr

In [12]:
import os

test_image_dir = "../data/vi/test_data/one_line"
test_images = os.listdir(test_image_dir)

for test_image in test_images:
    img_path = os.path.join(test_image_dir, test_image)

    # 🔄 Reuse the function
    final_text = get_transcript_from_image(img_path, reader, detector)

    print(f"{test_image}: {final_text if final_text else '(empty)'}")

04_001.png: VÌ CON LÀ
05_004.png: BÌNH TĨNH
